In [1]:
import xml.etree.ElementTree as ET

In [2]:
body = ET.parse('rltc.tmx').getroot().find('body')

In [3]:
clusters = []
for prop in list(body):
    translations = list(prop)
    texts = []
    for tran in translations:
        if tran.tag != 'prop':
            texts.append(list(tran)[0].text)
    clusters.append(texts)
            

In [4]:
clusters = [c[1:] for c in clusters if len(set(c)) > 2]

In [5]:
clusters[200]

['Безусловно, в Австрии, где корпоративные мероприятия не облагаются налогами, затраты на приглашения на фестивали Моцарта для любителей классической музыки или в Клостерс для лыжников обернутся большой выгодой.',
 'Безусловно, в Австрии, где расходы на корпоративные развлечения не облагаются налогами, проведение фестиваля Моцарта для любителей музыки или организация для сотрудников компании, увлекающихся горнолыжным спортом, отдыха на курорте Клостерс, обещает хорошую отдачу на вложенные в эти мероприятия средства.',
 'В Австрии, например, корпоративные мероприятия не облагаются налогами, поэтому билет на фестиваль Моцарта, подаренный ценителю музыки, вернется сторицей, а любители лыжного спорта оценят отдых на горнолыжном курорте Клостерс. Все это обещает хороший доход на первоначальные инвестиции.',
 'В Австрии, где корпоративные развлечения освобождаются от налогов, фестивали Моцарта для любителей музыки, и альпийский курорт для горнолыжников - хорошее вложение денег.']

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_checkpoint = 'cointegrated/rubert-base-cased-nli-twoway'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint)
if torch.cuda.is_available():
    model.cuda()

def predict_pair(text1, text2):
    with torch.inference_mode():
        out = model(**tokenizer(text1, text2, return_tensors='pt').to(model.device))
        proba = torch.softmax(out.logits, -1).cpu().numpy()[0]
        return {v: proba[k] for k, v in model.config.id2label.items()}['entailment']

In [ ]:
import tqdm
from itertools import permutations


cur_id = 0
for cluster in tqdm.tqdm(clusters[6186:]):
    cluster_id = cur_id
    for pair in permutations(set(cluster), 2):
        try:
            proba = predict_pair(*pair)
        except RuntimeError as e:
            print(e)
            continue
        data.append((cluster_id, *pair, proba))
    cur_id += 1

In [ ]:
len(data)

In [2]:
import pandas as pd

In [22]:

# df = pd.DataFrame(data, columns=['cluster_id', 'text1', 'text2', 'proba'])
df = pd.read_csv('data/rltc_entailment.csv')

In [23]:
len(df)

1214490

In [24]:
df.head()

,cluster_id,text1,text2,proba
0,0,"Однако, всего лишь один взгляд на структуру ОО...","Тем не менее, структура ООН предоставляет нам ...",0.016674
1,0,"Однако, всего лишь один взгляд на структуру ОО...","Однако, мгновенный взгляд на структуру ООН поз...",0.035338
2,0,"Однако, всего лишь один взгляд на структуру ОО...",Однако даже мимолетный взгляд на структуру ООН...,0.329359
3,0,"Однако, всего лишь один взгляд на структуру ОО...","Тем не менее, только один взгляд на структуру ...",0.590414
4,0,"Однако, всего лишь один взгляд на структуру ОО...",Хотя даже другой взгляд на структуру ООН дает ...,0.267942


In [25]:
df2 = df.copy()
df2.columns = ['cluster_id', 'text2', 'text1', 'ent']

df = df.merge(df2, on = ['text1', 'text2'])

In [26]:
import numpy as np

dupes = pd.DataFrame(np.sort(df[['text1', 'text2']].values, axis=1),
                     index=df[['text1', 'text2']].index,
                     columns=df[['text1', 'text2']].columns)
df = df[~dupes.duplicated()]

In [27]:
df = df[['cluster_id_x', 'text1', 'text2', 'proba', 'ent']]
df.columns = ['cluster_id', 'text1', 'text2', 'ent1', 'ent2']

In [28]:
len(df)

606957

In [29]:
df = df[(df.ent1 > 0.7) & (df.ent2 > 0.7)]

In [30]:
len(df)

148052

In [31]:
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()


df = df[~(df.text1.apply(lambda x: ad.is_latin(x)))]
df = df[~(df.text2.apply(lambda x: ad.is_latin(x)))]

In [32]:
len(df)

123696

In [34]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [35]:
df['leven'] = df.apply(lambda x: similar(x['text1'], x['text2']), axis=1)

In [51]:
df = df[df.leven < 0.85]

In [ ]:
df

In [65]:
45 in range(45, 55)

True

In [69]:
df = df[(df.text1.str.len() + df.text2.str.len()) > 50]

In [70]:
len(df)

117362

In [77]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)
from collections import Counter
from razdel import tokenize

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

def match_ents(s1, s2):
    doc1, doc2 = Doc(s1), Doc(s2)
    
    doc1.segment(segmenter)
    doc2.segment(segmenter)
    
    doc1.tag_ner(ner_tagger)
    doc2.tag_ner(ner_tagger)
    
    if Counter([s.type for s in doc1.spans]) != Counter([s.type for s in doc2.spans]):
        return False
    
    numbers1 = set()
    numbers2 = set()
    
    for tok in doc1.tokens:
        if has_numbers(tok.text):
            numbers1.add(tok.text)
    
    for tok in doc2.tokens:
        if has_numbers(tok.text):
            numbers2.add(tok.text)
    
    if numbers1 != numbers2:
        return False
    
    return True
    

def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

def match_abbr(s1, s2):
    abbrs1 = set()
    abbrs2 = set()
    
    for tok in tokenize(s1):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs1.add(tok.text)
    for tok in tokenize(s2):
        if tok.text.isupper() and len(tok.text) > 1:
            abbrs2.add(tok.text)
    if abbrs1 == abbrs2:
        return True
    return False

def match_latin(s1, s2):
    lat1 = set()
    lat2 = set()
    
    for tok in tokenize(s1):
        if ad.islatin(tok.text):
            lat1.add(tok.text)
    for tok in tokenize(s2):
        if ad.islatin(tok.text):
            lat2.add(tok.text)

    if lat1 == lat2:
        return True
    return False

In [78]:
df = df[df.apply(lambda x: match_abbr(x['text1'], x['text2']), axis=1)]

In [79]:
len(df)

110789

In [81]:
import tqdm

ner_data = []
for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    if match_ents(row['text1'], row['text2']):
        ner_data.append(i)

100%|██████████| 110789/110789 [07:56<00:00, 232.30it/s]


In [104]:
df = df[df.index.isin(ner_data)]

In [105]:
len(ner_data)

85116

In [106]:
def same(s1, s2):
    toks1 = set([_.text for _ in tokenize(s1) if _.text.isalpha()])
    toks2 = set([_.text for _ in tokenize(s2) if _.text.isalpha()])
    
    if toks1 == toks2:
        return True
    return False

In [107]:
df = df[~df.apply(lambda x: same(x['text1'], x['text2']), axis=1)]

In [109]:
df.to_csv('data/clean/rltc_clean.tsv', sep='\t', index=False)

In [76]:
q = df[df.leven > 0.82].sample(1).iloc[0]
print(q['text1'])
print(q['text2'])

То, насколько сильное будет сопротивление, зависит от нескольких факторов: размер вашей руки, скорость машины и плотность воздуха.
В нашем случае сила сопротивления зависит от нескольких факторов: размер вашей руки, скорость движения машины и плотность воздуха.
